## 1. Import Required Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import h5py

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,
    SeparableConv2D, GlobalMaxPooling2D, BatchNormalization
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## 2. Configure Dataset Paths

Update these paths according to your local directory structure.

In [ ]:
# Dataset paths - update these to your local paths
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))  # Current directory

train_dir = os.path.join(BASE_DIR, 'train')
val_dir = os.path.join(BASE_DIR, 'val')
test_dir = os.path.join(BASE_DIR, 'test')

print(f"Train directory: {train_dir}")
print(f"Validation directory: {val_dir}")
print(f"Test directory: {test_dir}")

## 3. Load and Preprocess Dataset

We load chest X-ray images in grayscale and convert them to RGB for compatibility with VGG16 pretrained weights.

In [ ]:
batch_size = 32
img_size = (224, 224)

# Create raw datasets
train_raw = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)

val_raw = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)

test_raw = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    color_mode='grayscale'
)

# Get class names before transformation
class_names = train_raw.class_names
print(f"Classes: {class_names}")
print(f"Total Classes: {len(class_names)}")

In [ ]:
# Convert grayscale to RGB (required for VGG16 weight compatibility)
def convert_gray_to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)
    return image, label

AUTOTUNE = tf.data.AUTOTUNE

# Apply mapping + prefetch for performance
train_ds = train_raw.map(convert_gray_to_rgb).prefetch(buffer_size=AUTOTUNE)
val_ds = val_raw.map(convert_gray_to_rgb).prefetch(buffer_size=AUTOTUNE)
test_ds = test_raw.map(convert_gray_to_rgb).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Verify data shape
for images, labels in train_ds.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

## 4. Visualize Sample Images

In [ ]:
plt.figure(figsize=(10, 10))

for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'), cmap='gray')
        label_idx = labels[i].numpy()
        plt.title(class_names[label_idx])
        plt.axis('off')

plt.suptitle('Sample Training Images', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Dataset Statistics

In [ ]:
# Count images per class
class_counts = {}

for class_name in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, class_name)
    if os.path.isdir(folder_path):
        class_counts[class_name] = len(os.listdir(folder_path))

print("Image count per class (Training Set):")
for class_name, count in class_counts.items():
    print(f"  {class_name}: {count}")

# Visualize class distribution
plt.figure(figsize=(8, 5))
plt.bar(class_counts.keys(), class_counts.values(), color=['green', 'red'])
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Training Set Class Distribution')
plt.show()

## 6. Data Augmentation

To handle class imbalance and improve generalization, we apply data augmentation.

In [ ]:
# Data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2),
])

# Apply augmentation and optimize pipeline
train_dataset = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y),
    num_parallel_calls=AUTOTUNE
)

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 7. Compute Class Weights

Class weights help the model handle imbalanced data by penalizing mistakes on the minority class more heavily.

In [ ]:
# Calculate class weights for imbalanced data
class_names_list = ['NORMAL', 'PNEUMONIA']
image_counts = [class_counts.get('NORMAL', 1341), class_counts.get('PNEUMONIA', 3875)]

y = np.repeat(np.arange(len(class_names_list)), image_counts)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y),
    y=y
)

class_weights_dict = dict(enumerate(class_weights))
print(f"Computed Class Weights: {class_weights_dict}")

## 8. Build CNN Model Architecture

Our model uses:
- **Conv2D layers** (first 2 blocks): Standard convolutions initialized with VGG16 weights
- **SeparableConv2D layers**: Efficient depthwise separable convolutions
- **BatchNormalization**: For training stability
- **Dropout**: For regularization (0.7 and 0.5)

In [ ]:
def build_model():
    """
    Build a custom CNN for pneumonia detection.
    
    Architecture:
    - 2 Conv2D blocks (compatible with VGG16 pretrained weights)
    - 2 SeparableConv2D blocks with BatchNormalization
    - Dense layers with Dropout for classification
    
    Returns:
        keras.Model: Compiled model ready for training
    """
    input_img = Input(shape=(224, 224, 3), name='ImageInput')
    
    # Block 1 - Conv2D (VGG16 weights compatible)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2, 2), name='pool1')(x)
    
    # Block 2 - SeparableConv2D
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2, 2), name='pool2')(x)
    
    # Block 3 - SeparableConv2D with BatchNorm
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2, 2), name='pool3')(x)
    
    # Block 4 - SeparableConv2D with BatchNorm
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2, 2), name='pool4')(x)
    
    # Classification Head
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(1, activation='sigmoid', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

# Build the model
model = build_model()
model.summary()

## 9. Transfer Learning: Load VGG16 Pretrained Weights

We load VGG16 pretrained weights for the first two convolutional layers to benefit from features learned on ImageNet.

> **Note**: Download VGG16 weights file (`vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5`) before running this cell.

In [ ]:
# Load VGG16 pretrained weights for early layers
VGG16_WEIGHTS_PATH = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

if os.path.exists(VGG16_WEIGHTS_PATH):
    with h5py.File(VGG16_WEIGHTS_PATH, 'r') as f:
        layer_map = {
            'Conv1_1': 'block1_conv1',
            'Conv1_2': 'block1_conv2',
        }

        for your_layer_name, vgg_layer_name in layer_map.items():
            target_layer = model.get_layer(your_layer_name)

            if len(target_layer.weights) == 2:
                w = f[vgg_layer_name][f"{vgg_layer_name}_W_1:0"][()]
                b = f[vgg_layer_name][f"{vgg_layer_name}_b_1:0"][()]
                target_layer.set_weights([w, b])
                print(f"✅ Loaded VGG16 weights for {your_layer_name}")
            else:
                print(f"⚠️ Skipping {your_layer_name} (incompatible layer type)")
    print("\n✅ VGG16 transfer learning weights loaded successfully!")
else:
    print(f"⚠️ VGG16 weights file not found at: {VGG16_WEIGHTS_PATH}")
    print("Proceeding with random initialization for Conv layers.")

## 10. Compile Model

In [ ]:
# Compile the model
optimizer = Adam(learning_rate=0.0001)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath='best_model_weights.weights.h5',
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss'
)

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy']
)

print("✅ Model compiled successfully!")

## 11. Train the Model

In [ ]:
# Train the model
EPOCHS = 20

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[early_stopping, checkpoint],
    class_weight={0: 1.0, 1: 0.4}  # Adjusted class weights
)

print("\n✅ Training complete!")

## 12. Training History Visualization

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
axes[0].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[0].set_title('Model Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True)

# Loss plot
axes[1].plot(history.history['loss'], label='Training Loss', marker='o')
axes[1].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[1].set_title('Model Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 13. Evaluate on Test Set

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"🧪 Test Loss: {test_loss:.4f}")

## 14. Confusion Matrix & Classification Report

In [ ]:
# Get predictions
y_true = []
y_pred = []

for images, labels in test_ds:
    y_true.extend(labels.numpy())
    preds = model.predict(images, verbose=0)
    y_pred.extend((preds > 0.5).astype("int").flatten())

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Pneumonia'],
            yticklabels=['Normal', 'Pneumonia'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Normal', 'Pneumonia']))

In [ ]:
# Calculate precision and recall
tn, fp, fn, tp = cm.ravel()

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")
print(f"\n📊 Precision: {precision:.4f}")
print(f"📊 Recall (Sensitivity): {recall:.4f}")
print(f"📊 F1-Score: {f1_score:.4f}")

## 15. Save the Trained Model

In [ ]:
# Save model in Keras format (required for Grad-CAM)
model.save('resume_model.keras')
print("✅ Model saved as 'resume_model.keras'")

## 16. Convert to TensorFlow Lite (for Deployment)

In [ ]:
# Convert to TFLite for efficient deployment
print("Converting to TFLite...")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite model saved as 'model.tflite'")
print(f"📦 TFLite model size: {os.path.getsize('model.tflite') / (1024*1024):.2f} MB")

---

## Summary

This notebook demonstrates the complete training pipeline for a pneumonia detection model:

1. **Data Loading**: Loaded chest X-ray images from train/val/test directories
2. **Preprocessing**: Converted grayscale to RGB, applied data augmentation
3. **Model Architecture**: Custom CNN with VGG16 transfer learning
4. **Training**: Used Adam optimizer, early stopping, and class weights
5. **Evaluation**: Assessed performance using accuracy, precision, recall, and F1-score
6. **Export**: Saved model in Keras format (for Grad-CAM) and TFLite (for deployment)

### Output Files
- `resume_model.keras` - Full Keras model (for Grad-CAM explainability)
- `model.tflite` - Optimized TFLite model (for fast inference)

### Next Steps
- Run the Streamlit app (`chest_xray/app.py`) to test the model with Grad-CAM visualization
- Fine-tune hyperparameters for better performance
- Experiment with different augmentation strategies